In [1]:
%reload_ext autoreload
%autoreload 2

In [13]:
import bleu
import weighted_ngram_match
import syntax_match
import dataflow_match

In [14]:
def compute_metrics(targets, suggestions):
    # calculate ngram match (BLEU)
    tokenized_targets = [x.split() for x in targets]
    tokenized_suggestions = [[x.split() for x in sugg] for sugg in suggestions]

    bleu_ngram_match_score = bleu.corpus_bleu(tokenized_suggestions, tokenized_targets)

    # calculate weighted ngram match
    keywords = [x.strip() for x in open(
        f'data/java-keywords.txt', 'r', encoding='utf-8').readlines()]


    def make_weights(reference_tokens, key_word_list):
        return {token: 1 if token in key_word_list else 0.2
                for token in reference_tokens}


    tokenized_suggs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]
                                    for reference_tokens in sugg] for sugg in tokenized_suggestions]

    weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(
        tokenized_suggs_with_weights, tokenized_targets)
    
    # calculate syntax match
    syntax_match_score = syntax_match.corpus_syntax_match(
        suggestions, targets, 'java')

    # calculate dataflow match
    dataflow_match_score = dataflow_match.corpus_dataflow_match(
        suggestions, targets, 'java')
    
    # return all scores in a dict
    return {
        'bleu_ngram_match': bleu_ngram_match_score,
        'weighted_ngram_match': weighted_ngram_match_score,
        'syntax_match': syntax_match_score,
        'dataflow_match': dataflow_match_score
    }

In [15]:
"""
public POIFSFileSystem(){
    this.phase = true;
    _header.BATCount = 1;
    _header.BATArray = (new int[]){1 };
    BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);
    bb.WorldBlockIndex=1;
    _bold_blocks.Add(bb);
    SetNextBlock(0, POIFSConstants.END_OF_CHAIN);
    SetNextBlock(1, POIFSConstants.FAT_SECTOR_BLOCK);
    _property_table.SetStartBlock(0);
}
"""

'\npublic POIFSFileSystem(){\n    this.phase = true;\n    _header.BATCount = 1;\n    _header.BATArray = (new int[]){1 };\n    BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);\n    bb.WorldBlockIndex=1;\n    _bold_blocks.Add(bb);\n    SetNextBlock(0, POIFSConstants.END_OF_CHAIN);\n    SetNextBlock(1, POIFSConstants.FAT_SECTOR_BLOCK);\n    _property_table.SetStartBlock(0);\n}\n'

# Correct suggestion

In [16]:
targets = ["""BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);"""]

suggestions = [["""BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);"""]]

compute_metrics(targets, suggestions)

(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier)))))) ['(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier))))))', '(local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier)))))', '(variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier))))', '(method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier)))', '(argument_list (false) (identifier))']


{'bleu_ngram_match': 1.0,
 'weighted_ngram_match': 1.0,
 'syntax_match': 1.0,
 'dataflow_match': 1.0}

# Suggestion swap variables

In [21]:
targets = ["""BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);"""]

suggestions = [["""BATBlock bb = BATBlock.CreateEmptyBATBlock(bigBlockSize, false);"""]]

compute_metrics(targets, suggestions)

(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false))))))
['(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier))))))', '(local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier)))))', '(variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier))))', '(method_invocation object: (identifier) name: (identifier) arguments: (argument_list (false) (identifier)))', '(argument_list (false) (identifier))']


{'bleu_ngram_match': 0.26591479484724945,
 'weighted_ngram_match': 0.26591479484724945,
 'syntax_match': 0.0,
 'dataflow_match': 1.0}

# Suggestion swap lines

In [22]:
targets = ["""
BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);
_header.BATArray = (new int[]){1 };
"""]

suggestions = [["""
_header.BATArray = (new int[]){1 };
BATBlock bb = BATBlock.CreateEmptyBATBlock(false, bigBlockSize);
"""]]

compute_metrics(targets, suggestions)

{'bleu_ngram_match': 0.785629301801026,
 'weighted_ngram_match': 0.785629301801026,
 'syntax_match': 0.9285714285714286,
 'dataflow_match': 0.5}

# Wrong binary operator

In [35]:
targets = ["""
BATBlock bb = cc + DD;
"""]

suggestions = [["""
BATBlock bb = cc * DD;
"""]]

compute_metrics(targets, suggestions)

(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (binary_expression left: (identifier) right: (identifier)))))
['(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (binary_expression left: (identifier) right: (identifier)))))', '(local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (binary_expression left: (identifier) right: (identifier))))', '(variable_declarator name: (identifier) value: (binary_expression left: (identifier) right: (identifier)))', '(binary_expression left: (identifier) right: (identifier))']

(local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (binary_expression left: (identifier) right: (identifier))))
['(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifie

{'bleu_ngram_match': 0.537284965911771,
 'weighted_ngram_match': 0.537284965911771,
 'syntax_match': 1.0,
 'dataflow_match': 1.0}

# Wrong type

In [34]:
targets = ["""
BATBlock bb = BATBlock.CreateEmptyBATBlock(bigBlockSize, false);
"""]

suggestions = [["""
float bb = BATBlock.CreateEmptyBATBlock(bigBlockSize, false);
"""]]

compute_metrics(targets, suggestions)

(program (local_variable_declaration type: (floating_point_type) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false))))))
['(program (local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false))))))', '(local_variable_declaration type: (type_identifier) declarator: (variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false)))))', '(variable_declarator name: (identifier) value: (method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false))))', '(method_invocation object: (identifier) name: (identifier) arguments: (argument_list (identifier) (false)))', '(argument_list (identifier) (false)

{'bleu_ngram_match': 0.668740304976422,
 'weighted_ngram_match': 0.5773502691896258,
 'syntax_match': 0.5,
 'dataflow_match': 1.0}

# Different naming but same origin

In [36]:
targets = ["""
float cc = 5;
float bb = cc - DD;
"""]

suggestions = [["""
float ii = 5;
float bb = ii - DD;
"""]]

compute_metrics(targets, suggestions)

<Node type=program, start_point=(0, 0), end_point=(1, 19)>
<Node type=local_variable_declaration, start_point=(1, 0), end_point=(1, 19)>
<Node type=variable_declarator, start_point=(1, 6), end_point=(1, 18)>
<Node type=binary_expression, start_point=(1, 11), end_point=(1, 18)>
<Node type=floating_point_type, start_point=(1, 0), end_point=(1, 5)>
<Node type=local_variable_declaration, start_point=(0, 0), end_point=(0, 13)>
<Node type=variable_declarator, start_point=(0, 6), end_point=(0, 12)>
<Node type=floating_point_type, start_point=(0, 0), end_point=(0, 5)>
<Node type=program, start_point=(0, 0), end_point=(1, 19)>
<Node type=local_variable_declaration, start_point=(1, 0), end_point=(1, 19)>
<Node type=variable_declarator, start_point=(1, 6), end_point=(1, 18)>
<Node type=binary_expression, start_point=(1, 11), end_point=(1, 18)>
<Node type=floating_point_type, start_point=(1, 0), end_point=(1, 5)>
<Node type=local_variable_declaration, start_point=(0, 0), end_point=(0, 13)>
<Node t

{'bleu_ngram_match': 0.4671379777282001,
 'weighted_ngram_match': 0.47960593523654194,
 'syntax_match': 1.0,
 'dataflow_match': 1.0}